### Importando dependências

In [69]:
import numpy as np
import pandas as pd

## Importação dos dados


In [70]:
dicionario = pd.read_csv('../datasets/dicionario_nascidos_vivos.csv')
df = pd.read_csv('../datasets/nascidos_vivos.csv')

In [71]:
df.head()

,Unnamed: 0,ano,sigla_uf,local_nascimento,data_nascimento,hora_nascimento,sexo,peso,raca_cor,id_anomalia,...,id_uf_mae,idade_mae,escolaridade_mae,estado_civil_mae,ocupacao_mae,raca_cor_mae,gestacoes_ant,quantidade_parto_normal,quantidade_parto_cesareo,idade_pai
0,0,2016,MT,1,2016-01-25,12:20:00,2,4235.0,1.0,2.0,...,50,37.0,4.0,5.0,NaN,1.0,3.0,1.0,2.0,NaN
1,1,2016,MT,1,2016-02-01,11:21:00,2,3145.0,1.0,2.0,...,51,27.0,4.0,5.0,NaN,1.0,2.0,0.0,2.0,NaN
2,2,2016,MT,1,2016-02-07,00:44:00,2,3315.0,4.0,2.0,...,51,25.0,4.0,2.0,NaN,4.0,0.0,0.0,0.0,NaN
3,3,2016,MT,1,2016-04-11,14:05:00,1,3275.0,4.0,2.0,...,51,27.0,4.0,2.0,NaN,4.0,3.0,3.0,0.0,NaN
4,4,2016,MT,1,2016-04-15,23:40:00,2,3500.0,2.0,2.0,...,26,16.0,4.0,5.0,NaN,2.0,1.0,1.0,0.0,21.0


In [72]:
dicionario.head()

,id_tabela,coluna,chave,cobertura_temporal,valor
0,microdados,local_nascimento,1,(1),Hospital
1,microdados,local_nascimento,2,(1),Outros estabelecimentos de saúde
2,microdados,local_nascimento,3,(1),Domicílio
3,microdados,local_nascimento,4,(1),Outros
4,microdados,local_nascimento,9,(1),Ignorado


## Limpeza e pré-processamento dos dados

  - Remover dados considerados desnecessários para o estudo.
  - Remover dados duplicados, caso existam.
  - Tratar os valores ausentes de acordo com a estratégia definida (ex: preenchimento com a média, exclusão de linhas ou colunas, etc.).

> Indented block


  - Verificar e tratar possíveis outliers ou dados inconsistentes.

### 1. Descartando colunas irrelevantes

As colunas relevantes já foram filtradas na consulta SQL inicial (Consulte o notebook [Download inicial dos dados]('') para mais detalhes). Sendo assim, eliminaremos apenas a coluna indexadora 

In [73]:
df = df.drop(['Unnamed: 0'], axis=1)

### 2. Tratando valores nulos

In [75]:
# Verificando presença de valores nulos
print(df.isnull().mean())

ano                         0.000000
sigla_uf                    0.000000
local_nascimento            0.000000
data_nascimento             0.000000
hora_nascimento             0.001368
sexo                        0.000000
peso                        0.000319
raca_cor                    0.034864
id_anomalia                 0.010260
codigo_anomalia             0.991345
semana_gestacao             0.017532
semana_gestacao_estimada    0.017531
gestacao_agr                0.016911
tipo_gravidez               0.001333
tipo_parto                  0.000806
inicio_pre_natal            0.029508
pre_natal                   0.019487
pre_natal_agr               0.000146
classificacao_pre_natal     0.000000
quantidade_filhos_vivos     0.037696
quantidade_filhos_mortos    0.058569
id_uf_mae                   0.016561
idade_mae                   0.000012
escolaridade_mae            0.007609
estado_civil_mae            0.006943
ocupacao_mae                0.245310
raca_cor_mae                0.041035
g

Podemos observar que 25 das 32 colunas possuem valores nulos ou faltantes. Em 6 colunas o volume ultrapassa 5%. Como temos muitos dados (mais de 17 milhões de linhas), aplicaremos estratégias de preenchimento apenas nessas 6 colunas. Os demais valores faltantes terão suas respectivas linhas removidas do dataframe.

In [82]:
df['codigo_anomalia'].fillna('-', inplace=True)
df['ocupacao_mae'].fillna('-', inplace=True)
df['idade_pai'].fillna(-1, inplace=True)
for column in ('quantidade_parto_cesareo', 'quantidade_parto_normal', 'quantidade_filhos_mortos'):
    df[column].fillna(df[column].mean(), inplace=True)

In [83]:
df = df.dropna()

In [84]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15170624 entries, 0 to 17322663
Data columns (total 31 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   ano                       int64  
 1   sigla_uf                  object 
 2   local_nascimento          int64  
 3   data_nascimento           object 
 4   hora_nascimento           object 
 5   sexo                      int64  
 6   peso                      float64
 7   raca_cor                  float64
 8   id_anomalia               float64
 9   codigo_anomalia           object 
 10  semana_gestacao           float64
 11  semana_gestacao_estimada  float64
 12  gestacao_agr              float64
 13  tipo_gravidez             float64
 14  tipo_parto                float64
 15  inicio_pre_natal          float64
 16  pre_natal                 float64
 17  pre_natal_agr             float64
 18  classificacao_pre_natal   int64  
 19  quantidade_filhos_vivos   float64
 20  quantidade_filhos_mortos  f

### 3. Removendo linhas duplicadas

In [85]:
df.duplicated().sum()

477

In [86]:
df = df.drop_duplicates()

### 4. Verificando dados inconsistentes

Aqui verificamos se há presença de dados inconsistentes nas principais colunas.

In [87]:
df['ano'].unique()

array([2016, 2019, 2018, 2015, 2017, 2020])

In [88]:
df['sigla_uf'].unique()

array(['MT', 'PB', 'ES', 'MS', 'AM', 'GO', 'AP', 'AC', 'RR', 'SC', 'RS',
       'PE', 'CE', 'MA', 'PA', 'SP', 'PR', 'TO', 'RO', 'MG', 'BA', 'RJ',
       'SE', 'DF', 'AL', 'RN', 'PI'], dtype=object)

In [89]:
df['sexo'].unique()

array([2, 1, 0])

In [94]:
df[df['peso'] <= 0]

,ano,sigla_uf,local_nascimento,data_nascimento,hora_nascimento,sexo,peso,raca_cor,id_anomalia,codigo_anomalia,...,id_uf_mae,idade_mae,escolaridade_mae,estado_civil_mae,ocupacao_mae,raca_cor_mae,gestacoes_ant,quantidade_parto_normal,quantidade_parto_cesareo,idade_pai
4906756,2015,AM,3,2015-03-04,10:30:00,2,0.0,5.0,2.0,-,...,13.0,29.0,2.0,1.0,-,5.0,1.0,1.0,0.0,43.0
8184201,2015,AP,5,2015-09-12,04:00:00,1,0.0,5.0,2.0,-,...,16.0,26.0,2.0,5.0,-,5.0,5.0,4.0,0.0,29.0
16468008,2015,AM,5,2015-03-02,17:00:00,2,0.0,5.0,2.0,-,...,13.0,21.0,3.0,1.0,622005.0,5.0,4.0,4.0,0.0,30.0


In [97]:
df = df[df['peso'] > 0]

In [98]:
df['tipo_parto'].unique()

array([2., 1., 9.])

In [100]:
df['tipo_gravidez'].unique()

array([1., 2., 3., 9.])

In [106]:
np.sort(df['idade_mae'].unique())

array([ 8.,  9., 10., 11., 12., 13., 14., 15., 16., 17., 18., 19., 20.,
       21., 22., 23., 24., 25., 26., 27., 28., 29., 30., 31., 32., 33.,
       34., 35., 36., 37., 38., 39., 40., 41., 42., 43., 44., 45., 46.,
       47., 48., 49., 50., 51., 52., 53., 54., 55., 56., 57., 58., 59.,
       60., 61., 62., 63., 64., 65., 99.])

In [109]:
df = df[df['idade_mae'] < 99]

Detectamos valores inválidos para as colunas `peso` (3 linhas) e `idade_mae` (13 linhas). Esses registros foram removidos.

### 5. Transformando tipos de dados

In [110]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15170131 entries, 0 to 17322663
Data columns (total 31 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   ano                       int64  
 1   sigla_uf                  object 
 2   local_nascimento          int64  
 3   data_nascimento           object 
 4   hora_nascimento           object 
 5   sexo                      int64  
 6   peso                      float64
 7   raca_cor                  float64
 8   id_anomalia               float64
 9   codigo_anomalia           object 
 10  semana_gestacao           float64
 11  semana_gestacao_estimada  float64
 12  gestacao_agr              float64
 13  tipo_gravidez             float64
 14  tipo_parto                float64
 15  inicio_pre_natal          float64
 16  pre_natal                 float64
 17  pre_natal_agr             float64
 18  classificacao_pre_natal   int64  
 19  quantidade_filhos_vivos   float64
 20  quantidade_filhos_mortos  f

In [113]:
# Convertendo 'object' para 'data'
df['data_nascimento'] = pd.to_datetime(df['data_nascimento'], format='%Y-%m-%d')
df['hora_nascimento'] = pd.to_datetime(df['hora_nascimento'], format='%H:%M:%S').dt.time

In [120]:
# Convertendo 'float64' para 'int64'
columns_to_convert = ['raca_cor', 'id_anomalia', 'semana_gestacao', 'semana_gestacao_estimada','gestacao_agr', 'tipo_gravidez', 'tipo_parto', 'inicio_pre_natal', 'pre_natal', 'pre_natal_agr','quantidade_filhos_vivos', 'quantidade_filhos_mortos', 'idade_mae', 'escolaridade_mae','estado_civil_mae', 'raca_cor_mae', 'gestacoes_ant', 'quantidade_parto_normal','quantidade_parto_cesareo', 'idade_pai']

for c in columns_to_convert:
    df[c] = df[c].astype('int64')

In [121]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15170131 entries, 0 to 17322663
Data columns (total 31 columns):
 #   Column                    Dtype         
---  ------                    -----         
 0   ano                       int64         
 1   sigla_uf                  object        
 2   local_nascimento          int64         
 3   data_nascimento           datetime64[ns]
 4   hora_nascimento           object        
 5   sexo                      int64         
 6   peso                      float64       
 7   raca_cor                  int64         
 8   id_anomalia               int64         
 9   codigo_anomalia           object        
 10  semana_gestacao           int64         
 11  semana_gestacao_estimada  int64         
 12  gestacao_agr              int64         
 13  tipo_gravidez             int64         
 14  tipo_parto                int64         
 15  inicio_pre_natal          int64         
 16  pre_natal                 int64         
 17  pre_natal_a

### 6. Salvando arquivo limpo

In [125]:
df.to_csv('../datasets/nascidos_vivos_limpo.csv', index=False)